In [9]:
import numpy as np


def make_batch(seq_data, word_dict, n_class):
    input_batch, target_batch = [], []

    for seq in seq_data:
        input = [word_dict[n] for n in seq[:-1]]  # 'm', 'a' , 'k' is input
        target = word_dict[seq[-1]]  # 'e' is target
        input_batch.append(np.eye(n_class)[input])
        target_batch.append(target)

    return input_batch, target_batch

In [10]:
import torch
import torch.nn as nn


class TextLSTM(nn.Module):
    def __init__(self, input_size, n_hidden, n_class):
        super(TextLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, n_hidden, batch_first=False)
        self.W = nn.Linear(n_hidden, n_class)
        self.b = nn.Parameter(torch.zeros(n_class))
        self.n_hidden = n_hidden

    def forward(self, X):
        input = X.transpose(0, 1)
        hidden_state = torch.zeros(1, len(X), self.n_hidden)
        cell_state = torch.zeros(1, len(X), self.n_hidden)

        outputs, (hidden_state, cell_state) = self.lstm(
            input, (hidden_state, cell_state)
        )
        outputs = outputs[-1]
        outputs = self.W(outputs) + self.b
        return outputs

In [11]:
import torch.optim as optim

if __name__ == '__main__':
    n_step = 3 # number of cells(= number of Step)
    n_hidden = 128 # number of hidden units in one cell

    char_arr = [c for c in 'abcdefghijklmnopqrstuvwxyz']
    word_dict = {n: i for i, n in enumerate(char_arr)}
    number_dict = {i: w for i, w in enumerate(char_arr)}
    n_class = len(word_dict)  # number of class(=number of vocab)

    seq_data = ['make', 'need', 'coal', 'word', 'love', 'hate', 'live', 'home', 'hash', 'star']

    model = TextLSTM(input_size=n_class, n_hidden=n_hidden, n_class=n_class)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    input_batch, target_batch = make_batch(seq_data,word_dict,n_class)
    input_batch = torch.FloatTensor(input_batch)
    target_batch = torch.LongTensor(target_batch)

    # Training
    for epoch in range(1000):
        optimizer.zero_grad()

        output = model(input_batch)
        loss = criterion(output, target_batch)
        if (epoch + 1) % 100 == 0:
            print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

        loss.backward()
        optimizer.step()

    inputs = [sen[:3] for sen in seq_data]

    predict = model(input_batch).data.max(1, keepdim=True)[1]
    print(inputs, '->', [number_dict[n.item()] for n in predict.squeeze()])

/tmp/ipykernel_3100812/2861800950.py:20: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  input_batch = torch.FloatTensor(input_batch)


Epoch: 0100 cost = 0.539015
Epoch: 0200 cost = 0.041550
Epoch: 0300 cost = 0.011508
Epoch: 0400 cost = 0.005084
Epoch: 0500 cost = 0.002877
Epoch: 0600 cost = 0.001862
Epoch: 0700 cost = 0.001307
Epoch: 0800 cost = 0.000970
Epoch: 0900 cost = 0.000750
Epoch: 1000 cost = 0.000598
['mak', 'nee', 'coa', 'wor', 'lov', 'hat', 'liv', 'hom', 'has', 'sta'] -> ['e', 'd', 'l', 'd', 'e', 'e', 'e', 'e', 'h', 'r']
